In [21]:
import os
import matplotlib.pyplot as plt

import librosa
import librosa.display

import IPython.display as ipd
import pandas as pd
import numpy as np
import random
import shutil
from tqdm.notebook import tqdm
import ipywidgets
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms


from PIL import Image
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure

Add Imports for Convolutional Nueral Network (CNN)

In [14]:
class CNNNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        #4 conv blocks / flatten/ linear/ softmax
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels = 3,
                out_channels = 8,
                kernel_size = 3,
                stride = 1,
                padding = 0,
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(
                in_channels = 8,
                out_channels = 16,
                kernel_size = 3,
                stride = 1,
                padding = 0,
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(
                in_channels = 16,
                out_channels = 32,
                kernel_size = 3,
                stride = 1,
                padding = 0,
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(
                in_channels = 32,
                out_channels = 64,
                kernel_size = 3,
                stride = 1,
                padding = 0,
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2)
        )
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(49152, 163)
        # self.sofmax = nn.Softmax()

    def forward(self, input_data):
        #print(input_data.shape, f'input_data {input_data.shape}')
        x = self.conv1(input_data)
        #print(x.shape, f'conv1 {x.shape}')
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.flatten(x)
        logits = self.linear(x)
        return logits


In [5]:
folder_path = os.path.join("fma_archive/fma_small/fma_small/")
audio_clips = []
folders = []

for folder in os.listdir(folder_path):
    if folder != "README.txt" and folder != "checksums":
        folders.append(folder)
        audio_clips = np.concatenate((os.listdir(folder_path + folder), audio_clips))
audio_clips

array(['155066.mp3', '154303.mp3', '154305.mp3', ..., '000995.mp3',
       '000997.mp3', '000998.mp3'], dtype='<U32')

In [6]:
def create_spectrogram(audio_path, audio_file):
    audio_file = os.path.join(audio_path, audio_file)

    #Load audio file
    x, sr = librosa.load(audio_file, sr=44100)

    #Display audio waveform
    # plt.figure(figsize=(14, 5))
    # plt.plot(x)

    #Create and display a spectogram with axis labels
    X = librosa.stft(x)
    Xdb = librosa.amplitude_to_db(abs(X))
    plt.figure(frameon=False)
    plt.figure(figsize=(8, 3))
    librosa.display.specshow(Xdb, sr=sr)
    #display(plt.gcf())
 
    #remove axes and title for PNG
    plt.title('')
    plt.axis('off')
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
    spectogram = plt.gcf()
    plt.close()
    spectogram.savefig('test_sounds/test_spectogram.png', transparent=True, bbox_inches='tight', pad_inches=0.0)
    return spectogram

In [20]:
def pre_process(current_folder, sound_file):
    spectogram = create_spectrogram(current_folder, sound_file)
    
    transform = transforms.Compose([
        transforms.Resize((800, 300)),  # Resize the image to match the model's input size
        transforms.ToTensor(),           # Convert the image to a tensor
    ])

    image = Image.open('test_sounds/test_spectogram.png')
    image = image.convert('RGB')
    input_tensor = transform(image)

    print(input_tensor.shape)

    return input_tensor

Distributing data Training, Validation and Test Folders

In [8]:
source_dir = 'spectograms'
train_dir = 'spectograms/training_data'
test_dir = 'spectograms/test_data'
val_dir = 'spectograms/validation_data'

Function to Distribute Training Data Set

In [ ]:
def distribute_files(train_split, test_split, files, source_dir, train_dir, test_dir, val_dir):
    # Move files to training set
    for f in files[:train_split]:
        shutil.move(os.path.join(source_dir, f), os.path.join(train_dir, f))

    # Move files to test set
    for f in files[train_split:train_split + test_split]:
        shutil.move(os.path.join(source_dir, f), os.path.join(test_dir, f))

    # Move files to validation set
    for f in files[train_split + test_split:]:
        shutil.move(os.path.join(source_dir, f), os.path.join(val_dir, f))

In [24]:
def copy_track_by_genres(directory, tracks_df):
    import ast
    for filename in os.listdir(directory):
        if filename.endswith(".png"):
            track_id = filename.split("_")[0].lstrip("0")
            temp = tracks_df.loc[tracks_df['track_id'] == track_id, 'genres_all']
            if not temp.empty:
                genres_for_track = tracks_df.loc[tracks_df['track_id'] == track_id, 'genres_all'].values[0]
                if not pd.isna(genres_for_track): # ensure genres are not empty
                    genres_list = ast.literal_eval(genres_for_track)
                    for genre in genres_list:
                        genre_folder = os.path.join(directory, str(genre))
                        os.makedirs(genre_folder, exist_ok=True)
                        shutil.copy2(os.path.join(directory, filename), os.path.join(genre_folder, filename))
                else:
                    print(f"Track {track_id} has no genres")

In [10]:
def distribute_files(source_dir, train_dir, test_dir, val_dir):
    files = os.listdir(source_dir)
    random.shuffle(files)

    total_files = len(files)
    train_size = int(0.8 * total_files)
    test_size = int(0.1 * total_files)

    for directory in [train_dir, test_dir, val_dir]:
        os.makedirs(directory, exist_ok=True)

    # Move files to training folder
    for filename in files[:train_size]:
        if filename.endswith(".png"):
            source_path = os.path.join(source_dir, filename)
            dest_path = os.path.join(train_dir, filename)
            shutil.move(source_path, dest_path)

    # Move files to testing folder
    for filename in files[train_size:train_size + test_size]:
        if filename.endswith(".png"):
            source_path = os.path.join(source_dir, filename)
            dest_path = os.path.join(test_dir, filename)
            shutil.move(source_path, dest_path)

    # Move files to validation folder
    for filename in files[train_size + test_size:]:
        if filename.endswith(".png"):
            source_path = os.path.join(source_dir, filename)
            dest_path = os.path.join(val_dir, filename)
            shutil.move(source_path, dest_path)

Move Files into Training, Test and Validation Sets

In [36]:
tracks = pd.read_csv('fma_archive/fma_metadata/tracks.csv')
genres = pd.read_csv('fma_archive/fma_metadata/genres.csv')
genres_df = genres[['genre_id', 'title']]
tracks_df = tracks[['track_id', 'genres_all', 'title']]
tracks_df.loc[tracks_df['track_id'] == 14063, 'genres_all']

# distribute_files(source_dir, train_dir, test_dir, val_dir)

C:\Users\Chomba Mumba\AppData\Local\Temp\ipykernel_25444\1476310598.py:1: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,48,49,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103) have mixed types. Specify dtype option on import or set low_memory=False.
  tracks = pd.read_csv('fma_archive/fma_metadata/tracks.csv')


8260    [185, 15]
Name: genres_all, dtype: object

In [27]:
copy_track_by_genres(val_dir, tracks_df)

Track 6674 has no genres
Track 6680 has no genres
Track 46043 has no genres


In [ ]:
files = [f for f in os.listdir(source_dir) if f.endswith('.png')] #file is every file in the spectograms directory

random.shuffle(files)
total_files = len(files)
train_split = int(0.8 * total_files)
test_split = int(0.1 * total_files)

distribute_files(train_split, test_split, files, source_dir, train_dir, test_dir, val_dir)

Data Loading

In [9]:

train_dataset = datasets.ImageFolder(
    train_dir,
    transforms.Compose([
        transforms.ToTensor(),
    ]))

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=25, shuffle=True, num_workers=0)

test_dataset = datasets.ImageFolder(
    test_dir,
    transforms.Compose([
        transforms.ToTensor(),
    ]))

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=25, shuffle=True, num_workers=0)

val_dataset = datasets.ImageFolder(
    val_dir,
    transforms.Compose([
        transforms.ToTensor(),
    ]))

val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=25, shuffle=True, num_workers=0)


In [68]:


def resize_images_in_directory(input_dir, target_size):
    # Iterate through subdirectories in the input directory
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith(('.png')):  # Adjust the file extensions as needed
                input_path = os.path.join(root, file)

                # Open the image
                with Image.open(input_path) as img:
                    # Resize the image
                    if img.size == target_size:
                        print(f"Skipping {input_path} as it's already resized to {target_size}")
                        continue  # Skip resizing for this image
                    img_resized = img.resize(target_size, Image.Resampling.LANCZOS)
                    # Overwrite the original image with the resized image
                    img_resized.save(input_path)


# Example usage
input_dir = 'input_directory'
output_dir = 'output_directory'
target_size = (800, 300)  # Set the target size for resizing

resize_images_in_directory(val_dir, target_size)



Skipping spectograms/validation_data\009307_spectogram.png as it's already resized to (800, 300)
Skipping spectograms/validation_data\009555_spectogram.png as it's already resized to (800, 300)
Skipping spectograms/validation_data\009560_spectogram.png as it's already resized to (800, 300)
Skipping spectograms/validation_data\009846_spectogram.png as it's already resized to (800, 300)
Skipping spectograms/validation_data\009887_spectogram.png as it's already resized to (800, 300)
Skipping spectograms/validation_data\010375_spectogram.png as it's already resized to (800, 300)
Skipping spectograms/validation_data\010384_spectogram.png as it's already resized to (800, 300)
Skipping spectograms/validation_data\010458_spectogram.png as it's already resized to (800, 300)
Skipping spectograms/validation_data\010697_spectogram.png as it's already resized to (800, 300)
Skipping spectograms/validation_data\011206_spectogram.png as it's already resized to (800, 300)
Skipping spectograms/validatio

In [ ]:
# folders.pop(0)
# for folder in folders[128:]:
#     current_folder = folder_path + folder  
#     for file in os.listdir(current_folder):
#         temp_file = file.replace('.mp3', '')
#         spectogram = create_spectrogram(current_folder, file)
#         spectogram.savefig(f'C:/Users/Chomba Mumba/Desktop/spectograms/{temp_file}_spectogram.png', transparent=True, bbox_inches='tight', pad_inches=0.0)
#         plt.clf()
#         plt.cla()
#         plt.close(spectogram)
#         del spectogram, temp_file

C:\Users\Chomba Mumba\AppData\Local\Temp\ipykernel_6632\567797151.py:15: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(8, 3))
C:\Users\Chomba Mumba\AppData\Local\Temp\ipykernel_6632\567797151.py:14: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(frameon=False)


KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

Create Training Function

In [10]:
def train(model, device, train_loader, validation_loader, epochs):
  criterion =  nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
  train_loss, validation_loss = [], []
  train_acc, validation_acc = [], []
  with tqdm(range(epochs), unit='epoch') as tepochs:
    tepochs.set_description('Training')
    for epoch in tepochs:
      model.train()
      # keep track of the running loss
      running_loss = 0.
      correct, total = 0, 0

      for data, target in train_loader:
        # getting the training set
        data, target = data.to(device), target.to(device)
        
        # Get the model output (call the model with the data from this batch)
        output = model(data)
        # Zero the gradients out)
        optimizer.zero_grad()
        # Get the Loss
        loss  = criterion(output, target)
        # Calculate the gradients
        loss.backward()
        # Update the weights (using the training step of the optimizer)
        optimizer.step()

        tepochs.set_postfix(loss=loss.item())
        running_loss += loss  # add the loss for this batch

        # get accuracy
        _, predicted = torch.max(output, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

      # append the loss for this epoch
      train_loss.append(running_loss.detach().cpu().item()/len(train_loader))
      train_acc.append(correct/total)

      # evaluate on validation data
      model.eval()
      torch.no_grad() #disable gradient calculation to improve efficency - Chomba
      running_loss = 0.
      correct, total = 0, 0

      for data, target in validation_loader:
        # getting the validation set
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        tepochs.set_postfix(loss=loss.item())
        running_loss += loss.item()
        # get accuracy
        _, predicted = torch.max(output, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

      validation_loss.append(running_loss/len(validation_loader))
      validation_acc.append(correct/total)

  return train_loss, train_acc, validation_loss, validation_acc

In [3]:
print(torch.cuda.get_device_name(0))

NVIDIA GeForce GTX 1650 Ti


In [15]:
cnn = CNNNetwork()

device  = 'cuda'
cnn.to(device)
train_loss, train_acc, validation_loss, validation_acc = train(cnn, device, train_loader, val_loader, 10)
train_loss, train_acc, validation_loss, validation_acc

  0%|          | 0/10 [00:00<?, ?epoch/s]

([3.8499031777707686,
  3.6238175860102873,
  3.498154160398874,
  3.4072872185559007,
  3.297913734957298,
  3.1091247937694098,
  2.8285740443638394,
  2.552470142056483,
  2.31703802073224,
  2.1385317974209044],
 [0.09894212818917238,
  0.13963907902924705,
  0.14971997510889856,
  0.16378344741754822,
  0.17535780958307406,
  0.18718108276291226,
  0.20323584318606097,
  0.2265090230242688,
  0.24816428126944617,
  0.2582451773490977],
 [5.449046977730685,
  6.036290202029916,
  5.9358278984247255,
  6.443776962368987,
  6.425990648047868,
  7.777574162150538,
  7.995044597359591,
  8.428510876589042,
  9.333352388337602,
  9.993577435959217],
 [0.010328638497652582,
  0.018779342723004695,
  0.009389671361502348,
  0.008450704225352112,
  0.012206572769953052,
  0.010328638497652582,
  0.016901408450704224,
  0.013145539906103286,
  0.018779342723004695,
  0.01971830985915493])

Map Class Names to index's

In [45]:
class_names = train_dataset.classes
class_mapping = {i: class_names[i] for i in range(len(class_names))}    

KeyError: '61'

In [30]:
def classify(model, current_folder, sound_file, device):
    model.eval()
    input_tensor = pre_process(current_folder, sound_file)

    input_tensor = input_tensor.to(device)

    input_tensor = input_tensor.unsqueeze(0)  # Adds a batch dimension of size 1

    print(input_tensor.shape)
    
    with torch.no_grad():
        output = model(input_tensor)
    
    probabilities = torch.nn.functional.softmax(output, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1).item()
    
    return predicted_class, probabilities.squeeze().tolist()

In [54]:
def map_classification_to_genre(classification, genres_df):
    predicted_label = class_mapping[int(classification)]
    return predicted_label

In [18]:
sound_file = 'test_sounds/001083.mp3'
current_folder = ''

In [44]:
predicted_class, probabilities = classify(cnn, current_folder, sound_file, 'cuda')
#predicted_class, probabilities = classify(cnn, current_folder, sound_file, device)
predicted_class, probabilities

torch.Size([3, 800, 300])
torch.Size([1, 3, 800, 300])


(61,
 [0.02796105481684208,
  0.025391431525349617,
  0.0001662766735535115,
  0.0001362029288429767,
  0.0015432650689035654,
  0.0008648903458379209,
  3.1413443934980023e-07,
  4.256294428017782e-09,
  0.003298285650089383,
  0.02256990782916546,
  0.0180276520550251,
  0.00011809293209807947,
  3.5357334127184004e-07,
  0.0016384421614930034,
  5.647815015663582e-08,
  0.18988893926143646,
  0.04862815514206886,
  0.03346336632966995,
  0.002797015942633152,
  0.0009056814596988261,
  0.0006709065055474639,
  0.0003187943948432803,
  0.009391676634550095,
  3.3855314995889785e-08,
  0.0011977116810157895,
  1.673673432378564e-05,
  0.011043877340853214,
  3.958568299822218e-07,
  3.5182194551452994e-05,
  0.0006244962569326162,
  5.383712505135918e-06,
  9.036839969667199e-07,
  0.003916594665497541,
  0.0044656433165073395,
  0.00039806682616472244,
  7.89872403594849e-11,
  0.00036226428346708417,
  0.002643306739628315,
  1.1599874394363496e-09,
  5.9028895549317895e-08,
  0.000

<Figure size 640x480 with 0 Axes>

In [55]:
genre = map_classification_to_genre(predicted_class, genres_df)

print(f"The predicted genre for {sound_file} is {genre}")

The predicted genre for test_sounds/001083.mp3 is 38
